#### 딥러닝에서도 기존의 머신러닝과 마찬가지로 하이퍼파라미터 탐색에 GridSearchCV나 RandomizedSearchCV를 활용할 수 있다.

###### ※딥러닝은 여타 머신러닝들과 달리 설정해야 할 하이퍼파라미터가 많기 때문에, GridSearchCV보다 RandomizedSearchCV가 유용하다.

In [1]:
|import tensorflow as tf
from tensorflow import keras
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(
    housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_full, y_train_full
)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [4]:
def build_model(n_hidden=1, n_neurons=30, learning_rate=3e-3, input_shape=[8]):
    model = keras.models.Sequential()
    model.add(keras.layers.InputLayer(input_shape=input_shape))
    for layer in range(n_hidden):
        model.add(keras.layers.Dense(n_neurons, activation="relu"))
    model.add(keras.layers.Dense(1))
    optimizer = keras.optimizers.SGD(learning_rate=learning_rate)
    model.compile(loss="mse", optimizer=optimizer)
    return model

In [5]:
keras_reg = keras.wrappers.scikit_learn.KerasRegressor(build_model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.
  """Entry point for launching an IPython kernel.


In [7]:
from scipy.stats import reciprocal
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

param_distribs = {
    "n_hidden": [0, 1, 2, 3],
    "n_neurons": np.arange(1, 100)               .tolist(),
    "learning_rate": reciprocal(3e-4, 3e-2)      .rvs(1000).tolist(),
}

rnd_search_cv = RandomizedSearchCV(keras_reg, param_distribs, n_iter=10, cv=3, verbose=2)
rnd_search_cv.fit(X_train, y_train, epochs=100,
                  validation_data=(X_valid, y_valid),
                  callbacks=[keras.callbacks.EarlyStopping(patience=10)])

Fitting 3 folds for each of 10 candidates, totalling 30 fits
Epoch 1/100
242/242 [==============================] - 2s 4ms/step - loss: 1.0422 - val_loss: 0.6167
Epoch 2/100
242/242 [==============================] - 1s 2ms/step - loss: 0.8901 - val_loss: 0.6140
Epoch 3/100
242/242 [==============================] - 1s 2ms/step - loss: 0.7213 - val_loss: 0.5144
Epoch 4/100
242/242 [==============================] - 1s 2ms/step - loss: 0.5473 - val_loss: 0.4867
Epoch 5/100
242/242 [==============================] - 1s 2ms/step - loss: 0.4819 - val_loss: 0.4711
Epoch 6/100
242/242 [==============================] - 0s 2ms/step - loss: 0.4656 - val_loss: 0.4538
Epoch 7/100
242/242 [==============================] - 1s 2ms/step - loss: 0.4529 - val_loss: 0.4391
Epoch 8/100
242/242 [==============================] - 1s 2ms/step - loss: 0.4393 - val_loss: 0.4312
Epoch 9/100
242/242 [==============================] - 1s 2ms/step - loss: 0.4341 - val_loss: 0.4216
Epoch 10/100
242/242 [========

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_search.py:972: UserWarning: One or more of the test scores are non-finite: [-3.21710298e-01 -1.65995339e+08 -4.40864275e-01 -3.90052249e-01
             nan -4.75872656e-01             nan -3.04575642e-01
 -3.06930194e-01 -3.15546801e-01]
  category=UserWarning,


363/363 [==============================] - 1s 2ms/step - loss: 0.9675 - val_loss: 0.5928
Epoch 2/100
363/363 [==============================] - 1s 2ms/step - loss: 0.5591 - val_loss: 0.5192
Epoch 3/100
363/363 [==============================] - 1s 2ms/step - loss: 0.5030 - val_loss: 0.4890
Epoch 4/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4906 - val_loss: 0.4587
Epoch 5/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4603 - val_loss: 0.4371
Epoch 6/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4473 - val_loss: 0.4255
Epoch 7/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4231 - val_loss: 0.4124
Epoch 8/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4126 - val_loss: 0.3993
Epoch 9/100
363/363 [==============================] - 1s 2ms/step - loss: 0.4006 - val_loss: 0.3917
Epoch 10/100
363/363 [==============================] - 1s 2ms/step - loss: 0.3914 - val_loss: 0.3862
E

RandomizedSearchCV(cv=3,
                   estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x7f2950f36110>,
                   param_distributions={'learning_rate': [0.0003238154147279126,
                                                          0.022237967826933876,
                                                          0.021861736200581385,
                                                          0.0025138794138987513,
                                                          0.008142341364172174,
                                                          0.001113310978659274,
                                                          0.009337511059623193,
                                                          0.0010697252060642905,
                                                          0.0003507235358974271,
                                                          0.0136653097624848...
                                                          0.01239259849

In [8]:
rnd_search_cv.best_params_

{'learning_rate': 0.009230840829704167, 'n_hidden': 2, 'n_neurons': 32}

In [9]:
rnd_search_cv.best_score_

-0.3045756419499715